In [ ]:
from tensorflow.keras.layers import Dense, Dropout, Conv2D, UpSampling2D, Flatten, Input, MaxPooling2D
from tensorflow.keras.activations import softmax, sigmoid
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img
import os

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import numpy as np
import sys
import tensorflow
print(tensorflow.__version__)

In [ ]:
# if running like python file.py
# base_path = os.path.abspath(os.path.dirname(__file__))
# print(base_path)

#if running in shell/jupyter
exec_path = os.getcwd()
print(exec_path)

In [ ]:
data_gen = ImageDataGenerator(rescale=1./255, validation_split=.3, 
                              rotation_range=90, horizontal_flip=True, 
                              vertical_flip=True, zca_whitening=False)
train_data_gen = data_gen.flow_from_directory(directory="C:/Users/akash/Downloads/keras tips and tricks/classify objects in video/Data/",
                             color_mode='rgb',
                             classes=['baseball', 'hockey'],
                             class_mode='categorical',
                             batch_size=32,
                             shuffle=False,
                             interpolation='nearest',
                             subset='training')

test_data_gen = data_gen.flow_from_directory(directory="C:/Users/akash/Downloads/keras tips and tricks/classify objects in video/Data/",
                                             target_size=(256,256),
                                             color_mode='rgb',
                                             classes=['baseball', 'hockey'],
                                             class_mode='categorical',
                                             batch_size=32,
                                             shuffle=False, interpolation='nearest',
                                             subset='validation')

In [ ]:
print("Training class info:\n Image shape: {}\n Class Indices: {}\n Class Mode: {}\n Data Format: {}\n".format(
    train_data_gen.image_shape, train_data_gen.class_indices, train_data_gen.class_mode, train_data_gen.data_format))
print(train_data_gen.num_classes)
print(train_data_gen.n)

In [ ]:
# We will use transfer learning to accelerate our training and model accuracy. 
# We are going to use ResNet 50 Layer model 

input_tensor = Input(shape=(256,256,3))
baseline_model = ResNet50(weights='imagenet', include_top=False, input_tensor= input_tensor)
head_model = baseline_model.output
head_model = MaxPooling2D(pool_size=(2,2))(head_model)
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(100, activation='relu')(head_model)
head_model = Dense(50, activation='relu')(head_model)
head_model = Dropout(0.4)(head_model)
head_model = Dense(train_data_gen.num_classes, activation='softmax')(head_model)

model = Model(inputs=baseline_model.input, outputs=head_model)

for layer in baseline_model.layers:
    layer.trainable = True

In [ ]:
input_tensor = Input(shape=(256,256,3))
def define_model(input_tensor):
    conv1 = Conv2D(32, kernel_size=(7,7), padding='same', activation='relu')(input_tensor)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    conv2 = Conv2D(64, kernel_size=(5,5), padding='same', activation='relu')(pool1)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    conv3 = Conv2D(128, kernel_size=(3,3), padding='same', activation='relu')(pool2)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    fully_connected = Flatten()(pool3)
    output = Dense(train_data_gen.num_classes, activation='softmax')(fully_connected)

    return output

model = Model(inputs= input_tensor, outputs= define_model(input_tensor))


In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data_gen, epochs=10, steps_per_epoch=train_data_gen.n//train_data_gen.batch_size, validation_data=test_data_gen, verbose=1, 
                    validation_steps=test_data_gen.n//test_data_gen.batch_size)


In [ ]:
model.save('C:/Users/akash/Downloads/keras tips and tricks/classify objects in video/Model/')

In [ ]:
import matplotlib.pyplot as plt 

plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])
plt.xlabel('epochs')
plt.legend(['val_accuracy', 'accuracy'])
plt.show()
plt.clf()

In [ ]:
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.xlabel('epochs')
plt.legend(['val_loss', 'loss'])
plt.show()
plt.clf()

Now we are going to test out model on video. 
For this to work, we need to 
1. use OpenCV so that we can extract the images from the videos.
2. take frame and pass it through the model.predict.
3. Show the output image with legend == predicted class

In [1]:
import cv2
from tensorflow.keras.models import load_model
import numpy as np

# mean = np.array([123.68, 116.779, 103.939], dtype='float32')
# print(mean)
label = ['baseball', 'hockey']
model = load_model('G:/My Drive/Model/')

video = cv2.VideoCapture('C:/Users/akash/Downloads/keras tips and tricks/classify objects in video/video/base_ball_test_video.mp4')
i = 0

while(True):
    ret, frames = video.read()
    if ret == False:
        print("Failed to read the video")
        break

    output = frames.copy()
    # cv2.imwrite('C:/Users/akash/Downloads/keras tips and tricks/classify objects in video/Data/baseball/image{}.png'.format(i), frames)
    frame_rgb = cv2.cvtColor(output, cv2.COLOR_BGR2RGB)
    resized_frame = np.resize(frame_rgb, new_shape=(1,256,256,3)).astype('float32')
    # resized_frame -= mean
    # print(resized_frame.shape)
    pred = model.predict(resized_frame, verbose=1)
    print(pred)
    
    class_indices = np.argmax(pred)
    print(class_indices)

    cv2.putText(output, label[class_indices], (35, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 255, 0), 5)
    
    cv2.imshow('frame',output)
#     i += 1
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video.release()
# cv2.destroyAllWindows()

1/1 [==============================] - 0s 3ms/step
[[9.9999917e-01 8.0583931e-07]]
0
1/1 [==============================] - 0s 1ms/step
[[9.9999917e-01 8.0765273e-07]]
0
1/1 [==============================] - 0s 2ms/step
[[9.9999917e-01 8.0693519e-07]]
0
1/1 [==============================] - 0s 998us/step
[[9.9999917e-01 8.0733849e-07]]
0
1/1 [==============================] - 0s 2ms/step
[[9.9999917e-01 8.0551428e-07]]
0
1/1 [==============================] - 0s 999us/step
[[9.9999917e-01 8.0323292e-07]]
0
1/1 [==============================] - 0s 987us/step
[[9.9999917e-01 7.9993356e-07]]
0
1/1 [==============================] - 0s 1000us/step
[[9.9999917e-01 7.9841152e-07]]
0
1/1 [==============================] - 0s 999us/step
[[9.9999917e-01 7.9831938e-07]]
0
1/1 [==============================] - 0s 977us/step
[[9.9999917e-01 8.0224379e-07]]
0
1/1 [==============================] - 0s 2ms/step
[[9.9999917e-01 8.0089075e-07]]
0
1/1 [==============================] - 0s 979us/step

1/1 [==============================] - 0s 1000us/step
[[9.9999988e-01 1.3345948e-07]]
0
1/1 [==============================] - 0s 1000us/step
[[9.9999988e-01 1.3345948e-07]]
0
1/1 [==============================] - 0s 999us/step
[[9.9999988e-01 1.3345948e-07]]
0
1/1 [==============================] - 0s 1ms/step
[[9.9999988e-01 1.3345948e-07]]
0
1/1 [==============================] - 0s 1ms/step
[[9.9999988e-01 1.3345948e-07]]
0
1/1 [==============================] - 0s 2ms/step
[[9.9999988e-01 1.3345948e-07]]
0
1/1 [==============================] - 0s 2ms/step
[[9.9999988e-01 1.3345948e-07]]
0
1/1 [==============================] - 0s 999us/step
[[9.9999988e-01 1.3345948e-07]]
0
1/1 [==============================] - 0s 1ms/step
[[9.9999988e-01 1.2565104e-07]]
0
1/1 [==============================] - 0s 994us/step
[[9.9999988e-01 1.2824799e-07]]
0
1/1 [==============================] - 0s 1ms/step
[[9.9999988e-01 1.2656241e-07]]
0
1/1 [==============================] - 0s 2ms/step
[[

1/1 [==============================] - 0s 2ms/step
[[7.2832624e-18 1.0000000e+00]]
1
1/1 [==============================] - 0s 2ms/step
[[6.8671463e-18 1.0000000e+00]]
1
1/1 [==============================] - 0s 969us/step
[[6.0460575e-18 1.0000000e+00]]
1
1/1 [==============================] - 0s 2ms/step
[[6.761713e-18 1.000000e+00]]
1
1/1 [==============================] - 0s 999us/step
[[5.3547533e-18 1.0000000e+00]]
1
1/1 [==============================] - 0s 2ms/step
[[3.852693e-18 1.000000e+00]]
1
1/1 [==============================] - 0s 2ms/step
[[5.9702873e-18 1.0000000e+00]]
1
1/1 [==============================] - 0s 2ms/step
[[5.6114547e-18 1.0000000e+00]]
1
1/1 [==============================] - 0s 2ms/step
[[5.1638578e-18 1.0000000e+00]]
1
1/1 [==============================] - 0s 2ms/step
[[4.698196e-18 1.000000e+00]]
1
1/1 [==============================] - 0s 999us/step
[[5.7805557e-18 1.0000000e+00]]
1
1/1 [==============================] - 0s 2ms/step
[[5.85776e-18 

1/1 [==============================] - 0s 2ms/step
[[9.9998939e-01 1.0640177e-05]]
0
1/1 [==============================] - 0s 967us/step
[[9.9998939e-01 1.0640177e-05]]
0
1/1 [==============================] - 0s 998us/step
[[9.9998939e-01 1.0640177e-05]]
0
1/1 [==============================] - 0s 999us/step
[[9.9998939e-01 1.0640177e-05]]
0
1/1 [==============================] - 0s 999us/step
[[9.9998939e-01 1.0640177e-05]]
0
1/1 [==============================] - 0s 1ms/step
[[9.9998736e-01 1.2677863e-05]]
0
1/1 [==============================] - 0s 2ms/step
[[9.9998724e-01 1.2739285e-05]]
0
1/1 [==============================] - 0s 2ms/step
[[9.99987006e-01 1.29917435e-05]]
0
1/1 [==============================] - 0s 999us/step
[[9.99987006e-01 1.29917435e-05]]
0
1/1 [==============================] - 0s 1000us/step
[[9.9998701e-01 1.2994606e-05]]
0
1/1 [==============================] - 0s 1ms/step
[[9.9998713e-01 1.2850248e-05]]
0
1/1 [==============================] - 0s 1ms/st

1/1 [==============================] - 0s 0s/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 967us/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 1000us/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 2ms/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 2ms/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 999us/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 1000us/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 1ms/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 2ms/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 2ms/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 1ms/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 1ms/step
[[0.9674259  0.03257413]]
0
1/1 [==============================] - 0s 9

...

[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 3ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - ETA:  - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 1000us/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 999us/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 3ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 1ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [=========================

1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 1ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 1ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 996us/step
[[0. 1.]]
1
1/1 [==============================] - 0s 999us/step
[[0. 1.]]
1
1/1 [==============================] - 0s 1000us/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 1ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 1ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 1000us/step
[[0. 1.]]
1
1/1 [==============================] - 0s 2ms/step
[[0. 1.]]
1
1/1 [==============================] - 0s 999

1/1 [==============================] - 0s 3ms/step
[[0.09760027 0.9023998 ]]
1
1/1 [==============================] - 0s 2ms/step
[[0.09680931 0.90319073]]
1
1/1 [==============================] - 0s 1ms/step
[[0.09680931 0.90319073]]
1
1/1 [==============================] - 0s 999us/step
[[0.09726425 0.90273577]]
1
1/1 [==============================] - 0s 3ms/step
[[0.09643476 0.9035653 ]]
1
1/1 [==============================] - 0s 999us/step
[[0.09643476 0.9035653 ]]
1
1/1 [==============================] - 0s 998us/step
[[0.09643476 0.9035653 ]]
1
1/1 [==============================] - 0s 2ms/step
[[0.09643476 0.9035653 ]]
1
1/1 [==============================] - 0s 2ms/step
[[0.09643476 0.9035653 ]]
1
1/1 [==============================] - 0s 2ms/step
[[0.09643476 0.9035653 ]]
1
1/1 [==============================] - 0s 1ms/step
[[0.09643476 0.9035653 ]]
1
1/1 [==============================] - 0s 1ms/step
[[0.09643476 0.9035653 ]]
1
1/1 [==============================] - 0s 2ms/

1/1 [==============================] - 0s 1ms/step
[[1.00000000e+00 1.01113896e-16]]
0
1/1 [==============================] - 0s 2ms/step
[[1.000000e+00 8.265404e-17]]
0
1/1 [==============================] - 0s 1000us/step
[[1.000000e+00 7.792402e-17]]
0
1/1 [==============================] - 0s 2ms/step
[[1.0000000e+00 1.2392988e-16]]
0
1/1 [==============================] - 0s 999us/step
[[1.0000000e+00 1.2255148e-16]]
0
1/1 [==============================] - 0s 1ms/step
[[1.0000000e+00 9.2130245e-17]]
0
1/1 [==============================] - 0s 1ms/step
[[7.9430427e-22 1.0000000e+00]]
1
1/1 [==============================] - 0s 2ms/step
[[8.416547e-22 1.000000e+00]]
1
1/1 [==============================] - 0s 2ms/step
[[8.529023e-22 1.000000e+00]]
1
1/1 [==============================] - 0s 994us/step
[[8.370664e-22 1.000000e+00]]
1
1/1 [==============================] - 0s 997us/step
[[8.562862e-22 1.000000e+00]]
1
1/1 [==============================] - 0s 1ms/step
[[8.544166e-22 

1/1 [==============================] - 0s 2ms/step
[[0.00248202 0.99751794]]
1
1/1 [==============================] - 0s 2ms/step
[[0.00248202 0.99751794]]
1
1/1 [==============================] - 0s 995us/step
[[0.00248202 0.99751794]]
1
1/1 [==============================] - 0s 2ms/step
[[0.00248202 0.99751794]]
1
1/1 [==============================] - 0s 1ms/step
[[0.00248202 0.99751794]]
1
1/1 [==============================] - 0s 2ms/step
[[0.00248202 0.99751794]]
1
1/1 [==============================] - 0s 3ms/step
[[0.00248202 0.99751794]]
1


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf

model = load_model('/gdrive/My Drive/Model/')

image = cv2.imread('/gdrive/My Drive/Data/baseball/00000493.jpg')
new_image = np.resize(image, (1, 256,256,3)).astype('float32')
@tf.function(experimental_relax_shapes=True)
def predict(x):
     return model.predict(x)
predicted_class = model.predict(new_image)
clas = np.argmax(predicted_class)
predicted_class

In [ ]:
clas